# Load dataset

In [28]:
# import library

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
# load dataset

x_train = pd.read_csv("x_train.csv")
y_train = pd.read_csv("y_train.csv")
x_val = pd.read_csv("x_val.csv")
y_val = pd.read_csv("y_val.csv")
x_test = pd.read_csv("x_test.csv")
y_test = pd.read_csv("y_test.csv")

In [30]:
# Convert y to NumPy arrays if still pandas Series
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()
y_test = y_test.to_numpy()
x_train = x_train.to_numpy()
x_val = x_val.to_numpy()
x_test = x_test.to_numpy()

# convert to 1D
y_train = y_train.ravel()
y_val = y_val.ravel()
y_test = y_test.ravel()

# Train model

In [31]:
model  = LogisticRegression(class_weight='balanced')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# Result model

In [32]:
# Compute metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


print("Results using sklearn LogisticRegression:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Results using sklearn LogisticRegression:
Accuracy: 0.6618
Precision: 0.2676
Recall: 0.6957
F1 Score: 0.3865


In [33]:
print('The details for confusion matrix is:')
print(classification_report(y_test, y_pred))

The details for confusion matrix is:
              precision    recall  f1-score   support

           0       0.92      0.66      0.77       636
           1       0.27      0.70      0.39       115

    accuracy                           0.66       751
   macro avg       0.60      0.68      0.58       751
weighted avg       0.82      0.66      0.71       751

